In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/srv/conda/envs/notebook/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.0.3.161/672/1 Dashboard: http://10.0.3.161:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,984,Ingrid,0.37614107378861306,0.0326675783585213
2000-01-01 00:00:01,963,Frank,-0.25542277892762844,-0.02808928177821035
2000-01-01 00:00:02,1030,Xavier,0.9177062414070785,-0.6940206995152813
2000-01-01 00:00:03,1015,Ursula,-0.148370673786131,-0.18146422880549729
2000-01-01 00:00:04,1009,Wendy,-0.3683559933164464,-0.3923673731140642
2000-01-01 00:00:05,980,Edith,0.2582115851661255,0.2744227907898813
2000-01-01 00:00:06,980,George,-0.515268329715955,0.18361066869925136
2000-01-01 00:00:07,960,Xavier,-0.8332451054838284,0.32516677549518014
2000-01-01 00:00:08,937,Wendy,-0.7632150137024083,0.8386605251057861


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,969,Xavier,-0.6111097161142283,-0.11656459420398768
2000-01-30 00:00:01,1081,Bob,0.2230327326823789,0.9589459081082023
2000-01-30 00:00:02,1026,Sarah,0.25292497990498886,-0.6441367967401748
2000-01-30 00:00:03,1000,Sarah,0.14901148095806382,0.6001264875413503
2000-01-30 00:00:04,1012,Quinn,0.2434241550899241,0.1726491609613785
2000-01-30 00:00:05,980,Michael,-0.9873008311638907,0.0941639776788421
2000-01-30 00:00:06,1007,Laura,0.10907673789208405,0.3430901973858258
2000-01-30 00:00:07,1006,Victor,-0.7657233424141741,0.2465661433291797
2000-01-30 00:00:08,993,Victor,-0.2719947896046002,-0.4529850803634019


In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,984,Ingrid,0.376141,0.032668
1,2000-01-01 00:00:01,963,Frank,-0.255423,-0.028089
2,2000-01-01 00:00:02,1030,Xavier,0.917706,-0.694021
3,2000-01-01 00:00:03,1015,Ursula,-0.148371,-0.181464
4,2000-01-01 00:00:04,1009,Wendy,-0.368356,-0.392367


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,984,Ingrid,0.376141,0.032668
1,2000-01-01 00:00:01,963,Frank,-0.255423,-0.028089
2,2000-01-01 00:00:02,1030,Xavier,0.917706,-0.694021
3,2000-01-01 00:00:03,1015,Ursula,-0.148371,-0.181464
4,2000-01-01 00:00:04,1009,Wendy,-0.368356,-0.392367


In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 5.19 s, sys: 535 ms, total: 5.72 s
Wall time: 4.12 s


name
Alice       0.002102
Bob        -0.002493
Charlie    -0.001104
Dan         0.002486
Edith      -0.002631
Frank       0.000913
George      0.000266
Hannah      0.002689
Ingrid      0.000663
Jerry      -0.000173
Kevin      -0.003633
Laura      -0.000632
Michael     0.000026
Norbert     0.000869
Oliver      0.000380
Patricia   -0.001270
Quinn       0.002163
Ray        -0.002743
Sarah      -0.000787
Tim         0.001211
Ursula      0.000994
Victor      0.000135
Wendy      -0.002047
Xavier      0.002100
Yvonne     -0.003877
Zelda      -0.001764
Name: x, dtype: float64

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.67 s, sys: 227 ms, total: 1.9 s
Wall time: 1.48 s


name
Alice       0.002102
Bob        -0.002493
Charlie    -0.001104
Dan         0.002486
Edith      -0.002631
Frank       0.000913
George      0.000266
Hannah      0.002689
Ingrid      0.000663
Jerry      -0.000173
Kevin      -0.003633
Laura      -0.000632
Michael     0.000026
Norbert     0.000869
Oliver      0.000380
Patricia   -0.001270
Quinn       0.002163
Ray        -0.002743
Sarah      -0.000787
Tim         0.001211
Ursula      0.000994
Victor      0.000135
Wendy      -0.002047
Xavier      0.002100
Yvonne     -0.003877
Zelda      -0.001764
Name: x, dtype: float64

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.45 s, sys: 153 ms, total: 1.6 s
Wall time: 1.25 s


name
Alice       0.002102
Bob        -0.002493
Charlie    -0.001104
Dan         0.002486
Edith      -0.002631
Frank       0.000913
George      0.000266
Hannah      0.002689
Ingrid      0.000663
Jerry      -0.000173
Kevin      -0.003633
Laura      -0.000632
Michael     0.000026
Norbert     0.000869
Oliver      0.000380
Patricia   -0.001270
Quinn       0.002163
Ray        -0.002743
Sarah      -0.000787
Tim         0.001211
Ursula      0.000994
Victor      0.000135
Wendy      -0.002047
Xavier      0.002100
Yvonne     -0.003877
Zelda      -0.001764
Name: x, dtype: float64